# Implementing Recommender Systems for BoardGamesGeek.com

In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
#line-by-line runtime comparison for easier code optimization.
%load_ext line_profiler

pd.set_option('display.max_rows',1000)

elite = pd.read_csv('../inputs/boardgame-elite-users.csv')
elite = elite.rename(columns = {'Compiled from boardgamegeek.com by Matt Borthwick':'UserID'})
titles = pd.read_csv('../inputs/boardgame-titles.csv')
titles = titles.rename(columns={'boardgamegeek.com game ID':'gameID'})
#frequent = pd.read_csv('../inputs/boardgame-frequent-users.csv')
#frequent = frequent.rename(columns = {'Compiled from boardgamegeek.com by Matt Borthwick':'UserID'})
#load up the big dataset
#users = pd.read_csv('../inputs/boardgame-users.csv')
#users = users.rename(columns = {'Compiled from boardgamegeek.com by Matt Borthwick':'UserID'})


## Baseline Predictor
This is the simplest predictor I'm making for the project. It doubles as a way to normalize the Ratings matrix R for more complex algorithms (like SVD) that require some kind of a way to fill missing ratings in the sparse matrix. Subtracting the baseline prediction from each value in R normalizes so that missing values can be set to 0.

All this prediction does is use the user rating averages, total average rating, and average item ratings to come up with a believable first guess. Details are in section 2.1 of the paper linked in the exploratory notebook. 

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split

class Base_Predictor(BaseEstimator,RegressorMixin):
    def __init__(self, DAMPENING_TERM=25, dampening=False):
        self._dampening_term = 25
        self._dampening = dampening
        
    def fit(self, X, y):
        self._mean = y.mean()
        self._R = pd.concat([X,y],axis=1).pivot_table(index='UserID',columns='gameID',values='rating')
        self._bu = self._R.apply(lambda row: self._user_base(row), axis=1)
        self._bi = self._R.apply(lambda column: self._item_base(column))
        
    def _user_base(self, row):
        """(1/M+d)*bu is with the dampening factor. Without it's 1/M * bu. To find the way to add the 
        dampening factor as a scalar multiplication: 
            k*1/M(bu) = 1/M+d(bu)
            k = M/M+d"""
        bu = row.mean() - self._mean
        if self._dampening:
            num_items_user_reviewed = row[row.notnull()].size
            damp_factor = num_items_user_reviewed/(num_items_user_reviewed+self._dampening_term)
            bu*=damp_factor
        return bu
    
    def _item_base(self, column):
        users_that_reviewed_this_item = column[column.notnull()]
        bu_for_users_that_reviewed_i = self._bu[users_that_reviewed_this_item.index].mean()
        bi = users_that_reviewed_this_item.mean()-bu_for_users_that_reviewed_i-self._mean
        if self._dampening:
            num_users_reviewed_item = column[column.notnull()].size
            damp_factor = num_users_reviewed_item/(num_users_reviewed_item+self._dampening_term)
            bi*=damp_factor
        return bi
        
    def predict(self, X):
        return self._mean + self._bu[X.UserID].values + self._bi[X.gameID].values
    

### A little Unit Testing for Sanity's sake.
I just made a simple 3x3 test data set with one missing value. I ran through the calculations by hand, and set up a little battery of tests to make sure it all works. I print out the 3 pieces of info so you can see visually. There's TDD_test_X, the user, item pair I predict. TDD_train_X is the list of values at the bottom, the middle matrix is TDD_test_X blown up into the ratings matrix (with the missing value I'm testing for showing). As long as this cell compiles without triggering an assertion error, things are working fine.

In [6]:
np.random.seed(42)

test_ratings_matrix = pd.DataFrame(np.random.randint(1,10,size=(3,3)),columns=map(int,list('456')),index=map(int,list('123')))
test_ratings_matrix.loc[3,6] = np.NaN
#collapse test frame down the same format as our dataset. 3 columns, user
TDD_train_X = test_ratings_matrix.stack().reset_index()
TDD_train_X.columns = ['UserID','gameID','rating']
TDD_test_X = pd.DataFrame(data={'UserID':[3],'gameID':[6]})
display(TDD_test_X)
display(test_ratings_matrix)
display(TDD_train_X)

def test_baseline_predictor():
    predictor = Base_Predictor()
    predictor.fit(TDD_train_X[['UserID','gameID']],TDD_train_X.rating)
    assert predictor._mean == 6.125, "The incorrect mean was calculated for the baseline test set"
    assert predictor._bu.tolist() == [0.20833333333333304,-1.125,1.375], "The wrong bu values were calculated for the baseline test"
    assert predictor._bi.tolist() == [0.05555555555555536, 0.05555555555555536, -0.16666666666666607], "incorrect bi was calculated for baseline test"
    assert predictor.predict(TDD_test_X).tolist() == [7.333333333333334], "baseline prediction for the test value was incorrect"
        

test_baseline_predictor()

,UserID,gameID
0,3,6


,4,5,6
1,7,4,8.0
2,5,7,3.0
3,7,8,NaN


,UserID,gameID,rating
0,1,4,7.0
1,1,5,4.0
2,1,6,8.0
3,2,4,5.0
4,2,5,7.0
5,2,6,3.0
6,3,4,7.0
7,3,5,8.0


## User-User Collaborative Filtering

This is the system described in section 2.2 of the linked paper. The idea is that to predict the rating of user U and item I, you use the normalized average rating of the N most similar users to U who have reviewed item I. There are multiple similarity measures that can be used, and several other hyper paramters that can be tweaked that can be fed into the class constructor for testing and comparison. 

In [176]:
from sklearn.base import BaseEstimator, RegressorMixin

class U_U_predictor(BaseEstimator, RegressorMixin):
    
    #ratings matrix from the actual training values
    #_R 
    
    #precomputing user's average ratings and std to save time later.
    #_user_average_rating
    #_user_standard_deviation
    
    #user similarity matrix (size user x user)
    #_S
    
    #function that changes depending on selected similarity metric (cosine, pearson, spearman, etc.)
    #_calculate_user_similarity
    
    #switches between equation 2.6 and 2.7 in the paper
    #_normalize_to_z_scores
    
    #the paper suggests a dampening threshhold to keep users from sparse reviews getting rated as overly similar
    #_pearson_threshold
    
    #how many nearest neighbors to look at when computing rating predictions
    #_N_similar
        
    def __init__(self, similarity_type = 'pearson', normalize_to_z_scores=False, pearson_threshold=50,N_similar=20):
        self._normalize_to_z_scores = normalize_to_z_scores
        self._pearson_threshold = pearson_threshold
        self._N_similar = N_similar
        if similarity_type=='cosine':
            self._calculate_user_similarity = self._cosine_similarity
        else: self._calculate_user_similarity = self._pearson_calculate_user_similarity
            
    def fit(self,X,y):
        self._R = pd.concat([X,y],axis=1).pivot_table(index='UserID',columns='gameID',values='rating')
        
        #preprocessing to make user similarities easier to calculate
        self._user_average_rating = self._R.mean(axis=1)
        self._user_standard_deviation = self._R.std(axis=1)
        self._calculate_item_lists(X)
        
        self._calculate_user_similarity_matrix_s()
        
    def _calculate_item_lists(self, X):
        """calculating user similarities is N^2, finding the intersection of two item lists from two users
        is a very costly part of that. To speed things up, we're going through and calculating that list 
        once per user, moving that side of things from O(N^2) to O(N). This saved ~50% runtime for the inner loop."""
        self._items_reviewed = {}
        
        for user in X.UserID.unique().tolist():
            ru = self._R.loc[user]
            self._items_reviewed[user] = set(ru[ru.notnull()].index)    

    def _calculate_user_similarity_matrix_s(self):    
        """All of the similarity measures I'm implementing are transitive (s(u,u') == s(u',u)) so I'm only calculating 
        one value for each pair once. If S is a UxU matrix, since you don't want to bother with calculating similarity 
        both for s(u,u') and s(u',u), that means we basically need to calculate values for only the upper triangle of 
        matrix S. We'll skip the main diagonal too, since we don't care about s(u,u). After calculating, we'll fill 
        out the matrix by mirroring the upper triangle down to the lower triangle to get our full matrix S."""
        
        self._S = pd.DataFrame(index=self._R.index, columns = self._R.index.rename('User_Prime_ID'), data=0.0)
        self._S = self._S.mask(np.tril(np.ones(self._S.shape, dtype=np.bool_)))
        user_combinations = self._S.stack().reset_index()
        user_combinations.columns = ['UserID','User_Prime_ID','rating']
        
        user_combinations.rating = user_combinations.apply(
            lambda row: self._calculate_user_similarity(row), axis=1)
        
        temp_matrix = user_combinations.pivot_table(index='UserID',columns='User_Prime_ID',values='rating').as_matrix()
        
        #adding in left column and bottom row of NaNs so we can remap our new values to the upper and lower triangle
        #of _S, and have all the indices match. 
        temp_matrix = np.append(np.full((temp_matrix.shape[0],1),np.NaN),temp_matrix,1)
        temp_matrix = np.append(temp_matrix,np.full((1,temp_matrix.shape[1]),np.NaN),0)
        
        #mirror the upper triangle of matrix S (that we have values to) to the lower triangle to complete our matrix.
        i_lower = np.tril_indices(temp_matrix.shape[0], -1)
        temp_matrix[i_lower] = temp_matrix.T[i_lower] 
        self._S = pd.DataFrame(temp_matrix, columns=self._S.columns, index=self._S.index)
        
    def _pearson_calculate_user_similarity(self, row):
        user1, user2 = row.UserID, row.User_Prime_ID
        items_in_common = self._find_items_two_users_both_reviewed(user1, user2)
        user1_average = self._user_average_rating[user1]
        user2_average = self._user_average_rating[user2]
        
        user1_normalized_ratings = self._R.loc[user1, items_in_common] - user1_average
        user2_normalized_ratings = self._R.loc[user2, items_in_common] - user2_average
        
        user1_norm = np.linalg.norm(user1_normalized_ratings)
        user2_norm = np.linalg.norm(user2_normalized_ratings)
        
        #Rating vectors with very low standard deviation have very low norms using this formula. I'm tossing them
        #out to prevent a divide by zero below. 
        if user1_norm < .0001 or user2_norm < .0001:
            return 0.0
        
        corr = np.dot(user1_normalized_ratings, user2_normalized_ratings)/(user1_norm * user2_norm)
        
        dampening_factor = min((len(items_in_common)/self._pearson_threshold), 1.0)

        return corr*dampening_factor

    def _cosine_similarity(self):
        pass
    
    def _find_N_similar_users(self, user1,user2):
        pass
    
    def _find_items_two_users_both_reviewed(self, user1,user2):
        return self._items_reviewed[user1].intersection(self._items_reviewed[user2])
    
    def predict(self,X):
        return X.apply(lambda row: self._single_prediction(row), axis=1).values
        
    def _single_prediction(self, row):
        user_mean = self._user_average_rating[row.UserID]
        N = self._N_similar
    
        #nearest users IDs are the index of this series, s(u,u') is the values. It's filtered so only users
        #who have also rated the game in question are considered. 
        N_nearest_similar = self._S[row.UserID][self._R[row.gameID].notnull()].nlargest(N)
    
        #vectors for most similar user information
        similar_users_means = self._user_average_rating[N_nearest_similar.index].values
        similar_users_item_ratings = self._R.loc[N_nearest_similar.index, row.gameID].values
    
        user_std = 1.0
        similar_users_stds = np.full(N,1.0)
        if(self._normalize_to_z_scores):
            user_std = self._user_standard_deviation[row.UserID]
            similar_users_stds = self._user_standard_deviation[N_nearest_similar.index].values
    
        similar_users_z_values = (similar_users_item_ratings-similar_users_means)/similar_users_stds
        prediction = user_mean + ((user_std/N) * np.dot(N_nearest_similar.values,similar_users_z_values))
    
        return prediction



In [177]:
train_X, test_x, train_y,test_y = train_test_split(elite[['UserID','gameID']],elite.rating,test_size=.3,random_state=42)

predictor = U_U_predictor()
predictor.fit(train_X,train_y)

## Model Selection and Error checking

Now that I've gotten some models built out, I can use Sklearn's framework to check out different prediction systems, compare RMSE, and see what kind of model works the best with this dataset.

In [178]:
from sklearn.metrics import mean_squared_error

#predictor = Base_Predictor()
#train_X, test_x, train_y,test_y = train_test_split(elite[['UserID','gameID']],elite.rating,test_size=.3,random_state=42)
#predictor.fit(train_X,train_y)

predictions = predictor.predict(test_x)

mse = mean_squared_error(predictions,test_y)
print(np.sqrt(mse))

1.32964534833


In [186]:
predictor = U_U_predictor(normalize_to_z_scores=True, N_similar=10)
predictor.fit(train_X,train_y)

predict = predictor.predict(test_X)

mse = mean_squared_error(predict,test_y)
print(np.sqrt(mse))

1.31049914233


In [184]:
predictor = U_U_predictor(normalize_to_z_scores=True, N_similar=5)
predictor.fit(train_X,train_y)

predict = predictor.predict(test_X)

mse = mean_squared_error(predict,test_y)
print(np.sqrt(mse))

1.30633111229


In [185]:
predictor = U_U_predictor(normalize_to_z_scores=True, N_similar=3)
predictor.fit(train_X,train_y)

predict = predictor.predict(test_X)

mse = mean_squared_error(predict,test_y)
print(np.sqrt(mse))

1.31054160406


In [ ]:
predictor = Base_Predictor()
predictor.fit(train_X,train_y)

predictions = predictor.predict(test_x)

mse = mean_squared_error(predictions,test_y)
print(np.sqrt(mse))